### **Import Reuqired Libraries**

In [51]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder,StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,classification_report,f1_score
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
import warnings
warnings.filterwarnings("ignore")

### **Import Datasets**

In [52]:
train_data=pd.read_csv("../dataset/train_data.csv")
test_data=pd.read_csv("../dataset/test_data.csv")

In [53]:
print(f"Shape of the Training data:{train_data.shape}")
print(f"Shape of the Testing data:{test_data.shape}")

Shape of the Training data:(1550, 13)
Shape of the Testing data:(388, 13)


### **Drop Unecessary Columns**

In [54]:
train_data=train_data=train_data.drop(["Unnamed: 0"],axis=1)
test_data=test_data.drop(["Unnamed: 0"],axis=1)

In [55]:
##split train data into the X_train and Y_train
X_train=train_data.drop(["delivery_status"],axis=1)
Y_train=train_data["delivery_status"]

In [56]:
##Adjust the datatype of  assembly_service_requested column
X_train["assembly_service_requested"]=X_train["assembly_service_requested"].astype("object")

### **Model Building**

In [57]:
# ### Numbercal columns 
numerical_features=X_train.select_dtypes(include="number").columns.to_list()
categorical_features=X_train.select_dtypes(include="object").columns.to_list()

###Nominal variables
categorical_features.remove("brand")
##Ordinal variables
ordinal_features=["brand"]


In [58]:
### Define pipelines
nominal_pipeline=Pipeline(steps=[
    ("One-Hot-Encoder",OneHotEncoder(sparse_output=False,handle_unknown="ignore"))
])
numerical_pipeline=Pipeline(steps=[
    ("Standard Scaler",StandardScaler())
])

ordinal_pipeline=Pipeline(steps=[
    ("Ordinal-Encoder",OrdinalEncoder())
])


transfomer=ColumnTransformer(transformers=[
    ("Numerical Pipeline",numerical_pipeline,numerical_features),
    ("Nominal Pipeline",nominal_pipeline,categorical_features),
    ("Ordinal Pipeline",ordinal_pipeline,ordinal_features)
])

final_pipeline=Pipeline(steps=[
    ("Transfomer",transfomer)
  
])

In [59]:
X_train_preprocessed = final_pipeline.named_steps["Transfomer"].fit_transform(X_train)

# Get the ColumnTransformer from the pipeline
preprocessor = final_pipeline.named_steps["Transfomer"]

# Get feature names from the ColumnTransformer
feature_names = preprocessor.get_feature_names_out()
X_train_preprocessed=pd.DataFrame(X_train_preprocessed,columns=feature_names)

In [60]:
##split train data into the X_train and Y_train
X_test=test_data.drop(["delivery_status"],axis=1)
Y_test=test_data["delivery_status"]

##Adjust the datatype of  assembly_service_requested column
X_test["assembly_service_requested"]=X_test["assembly_service_requested"].astype("object")

X_test_preprocessed = final_pipeline.named_steps["Transfomer"].transform(X_test)
X_test_preprocessed=pd.DataFrame(X_test_preprocessed,columns=feature_names)

In [61]:
##mapping y variable with the numeric values
y_map={"Failed Delivery":0,"On Going":1,"Delivered":2}
Y_train=Y_train.map(y_map)
Y_test=Y_test.map(y_map)

In [62]:
##implementing ANN using tensorflow
model=tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_preprocessed.shape[1],)), ##Input Layer
    tf.keras.layers.Dense(10,activation="relu"), ## First Hidden Layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(6,activation="relu"), ##Second Hidden Layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(8,activation="relu"), ##Third Hidden Layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(3,activation="softmax") ##Output Layer
])

model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.fit(X_train_preprocessed,Y_train,validation_data=(X_test_preprocessed,Y_test),epochs=200,batch_size=20,verbose=1)

Epoch 1/200


78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3439 - loss: 1.3289 - val_accuracy: 0.4768 - val_loss: 1.0578
Epoch 2/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3548 - loss: 1.2534 - val_accuracy: 0.4897 - val_loss: 1.0391
Epoch 3/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3813 - loss: 1.1789 - val_accuracy: 0.4923 - val_loss: 1.0298
Epoch 4/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4265 - loss: 1.1200 - val_accuracy: 0.4948 - val_loss: 1.0235
Epoch 5/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4303 - loss: 1.0978 - val_accuracy: 0.4948 - val_loss: 1.0183
Epoch 6/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4497 - loss: 1.0711 - val_accuracy: 0.4948 - val_loss: 1.0176
Epoch 7/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4419 - loss: 1.0708 - val_accuracy: 0.4948 - val_loss: 1.0177
Epoch 8/200
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4742 - loss: 1.0508 - val_accuracy: 0.4948 - val_loss: 1.0

### **Model Evaluation**

In [63]:
y_pred=model.predict(X_test_preprocessed)
y_pred

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


array([[0.4961761 , 0.32474014, 0.17908375],
       [0.49662533, 0.32676482, 0.17660987],
       [0.49437678, 0.3281341 , 0.17748913],
       ...,
       [0.49618226, 0.3248032 , 0.17901461],
       [0.4960771 , 0.32638842, 0.17753448],
       [0.4945066 , 0.3294646 , 0.1760289 ]],
      shape=(388, 3), dtype=float32)

In [64]:
y_pred=np.argmax(y_pred,axis=1)

In [65]:
y_pred_train=model.predict(X_train_preprocessed)
y_pred_train= np.argmax(y_pred_train, axis=1)

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [66]:
##accuracy,precision,recall,f1 score for training and testing data
accuracy_test=accuracy_score(Y_test,y_pred)
precision_test=precision_score(Y_test,y_pred,average="weighted")
recall_test=recall_score(Y_test,y_pred,average="weighted")
f1_test=f1_score(Y_test,y_pred,average="weighted")

accuracy_train = accuracy_score(Y_train,y_pred_train)
precision_train = precision_score(Y_train,y_pred_train,average="weighted")
recall_train = recall_score(Y_train,y_pred_train,average="weighted")
f1_train = f1_score(Y_train,y_pred_train,average="weighted")

In [67]:
##print precision,recall,f1-score for testing data
##print accuracy   
print(f"Accuracy of the ANN model:{accuracy_test}")
##print precision
print(f"Precision of the ANN model:{precision_test}")
##print recall
print(f"Recall of the ANN model:{recall_test}")
##print f1 score
print(f"F1 Score of the ANN model:{f1_test}")

Accuracy of the ANN model:0.4948453608247423
Precision of the ANN model:0.24487193112976938
Recall of the ANN model:0.4948453608247423
F1 Score of the ANN model:0.32762175613224315


In [68]:
##print the precision,recall,f1-score for training data
##print accuracy   
print(f"Accuracy of the ANN model:{accuracy_train}")
##print precision
print(f"Precision of the ANN model:{precision_train}")
##print recall
print(f"Recall of the ANN model:{recall_train}")
##print f1 score
print(f"F1 Score of the ANN model:{f1_train}")

Accuracy of the ANN model:0.49483870967741933
Precision of the ANN model:0.2448653485952133
Recall of the ANN model:0.49483870967741933
F1 Score of the ANN model:0.32761440683865395


In [69]:
##print classification report
print("Classification Report:")
print(classification_report(Y_test,y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.49      1.00      0.66       192
           1       0.00      0.00      0.00       127
           2       0.00      0.00      0.00        69

    accuracy                           0.49       388
   macro avg       0.16      0.33      0.22       388
weighted avg       0.24      0.49      0.33       388

